In [1]:
'''


我们将把过去几节中学到的 PyTorch 工作流程应用到计算机视觉中。


0.PyTorch中的计算机视觉库     PyTorch 有很多内置的有用的计算机视觉库，让我们来看看。
1. 加载数据                 为了练习计算机视觉，我们将从FashionMNIST中不同服装的一些图像开始
2. 准备数据                 我们有一些图像，让我们使用PyTorch DataLoader加载它们，以便我们可以在训练循环中使用它们。
3. 模型0: 构建基线模型       在这里，我们将创建一个多类分类模型来学习数据中的模式，我们还将选择损失函数、优化器并构建训练循环。
4. 进行预测和评估模型 0       让我们用基线模型进行一些预测并对其进行评估。
5. 为未来模型设置设备无关代码  最佳实践是编写与设备无关的代码，所以让我们来设置它。
6. 模型 1: 添加非线性        实验是机器学习的重要组成部分，让我们尝试通过添加非线性层来改进我们的基线模型。
7. 模型2: 卷积神经网络(CNN)  是时候具体了解计算机视觉并介绍强大的卷积神经网络架构了。
8. 比较我们的模型            我们构建了三种不同的模型，让我们对它们进行比较。
9. 评估我们的最佳模型        让我们对随机图像进行一些预测并评估我们的最佳模型。
10.制作混淆矩阵             混淆矩阵是评估分类模型的好方法，让我们看看如何制作一个。
11.保存和加载性能最佳的模型   由于我们可能想稍后使用我们的模型，因此我们保存它并确保它正确加载。

'''

'\n\n\n'

In [2]:
'''
0.PyTorch中的计算机视觉库

torchvision               包含常用于计算机视觉问题的数据集、模型架构和图像转换。
torchvision.datasets     在这里，您将找到许多示例计算机视觉数据集，用于解决图像分类、对象检测、图像字幕、视频分类等一系列问题。它还包含一系列用于制作自定义数据集的基类。
torchvision.models       该模块包含在 PyTorch 中实现的性能良好且常用的计算机视觉模型架构，您可以将它们用于解决您自己的问题。
torchvision.transforms   在与模型一起使用之前，图像通常需要进行转换（转换为数字/处理/增强），常见的图像转换可以在此处找到。
torch.utils.data.Dataset  PyTorch 的基础数据集类。
torch.utils.data.DataLoader     在数据集上创建 Python 可迭代对象（使用torch.utils.data.Dataset创建）。


注意： torch.utils.data.Dataset 和 torch.utils.data.DataLoader
类不仅适用于 PyTorch 中的计算机视觉，它们还能够处理许多不同类型的数据。


现在我们已经介绍了一些最重要的 PyTorch 计算机视觉库，让我们导入相关的依赖项。

'''

# Import PyTorch
import torch
from torch import nn

# Import torchvision 
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")



PyTorch version: 2.4.0+cu121
torchvision version: 0.19.0+cu121


In [3]:
'''
1. 获取数据集

我们将从 FashionMNIST 开始。
MNIST stands for Modified National Institute of Standards and Technology.
MNIST 代表修改后的国家标准与技术研究院。
原始 MNIST 数据集包含数千个手写数字示例（从 0 到 9），用于构建计算机视觉模型来识别邮政服务号码。
由 Zalando Research 开发的FashionMNIST也是类似的设置。
但它包含 10 种不同服装的灰度图像。

torchvision.datasets包含许多示例数据集，您可以使用它们来练习编写计算机视觉代码。 
FashionMNIST 就是其中之一。由于它有 10 个不同的图像类别（不同类型的服装），因此这是一个多类别分类问题。

PyTorch 在torchvision.datasets中存储了一堆常见的计算机视觉数据集。
将 FashionMNIST 纳入其中。

要下载它，我们提供以下参数：
root: str           - 您要将数据下载到哪个文件夹？
train: Bool         - 你想要训练还是测试分开？
download: Bool      - 是否应该下载数据？
transform: torchvision.transforms - 您想对数据进行什么转换？
target_transform    - 如果您也愿意，您可以转换目标（标签）。

'''

# Setup training data
train_data = datasets.FashionMNIST(
    root="data", # where to download data to?
    train=True, # get training data
    download=True, # download data if it doesn't exist on disk
    transform=ToTensor(), # images come as PIL format, we want to turn into Torch tensors
    target_transform=None # you can transform labels as well
)

# Setup testing data
test_data = datasets.FashionMNIST(
    root="data",
    train=False, # get test data
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:03<00:00, 7040153.72it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 91248.47it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:02<00:00, 2170359.71it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 11176126.81it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

